# A7. Matchups
Source: DKSalaries, A5. Stats <br>

Description: This uses DKSalaries and A5. Stats to create matchup Excel files as simulation input <br>

### Imports

In [1]:
import numpy as np
import pandas as pd
import os
import xlrd
import unidecode
import datetime
from datetime import date
from pathlib import Path
import os
import glob
import re
import time

# import import_ipynb
# from Utilities import *

import sys
sys.path.append(r"C:\Users\james\Documents\MLB\Code")
from Utilities import *

import warnings
warnings.simplefilter(action="ignore")

from distutils.dir_util import copy_tree

baseball_path = r"C:\Users\james\Documents\MLB\Data"
download_path = r"C:\Users\james\Downloads"

In [2]:
# Today's Date
# YYYY-MM-DD (datetime)
todaysdate_dt = datetime.date.today()

# YYYY-MM-DD (string)
todaysdate_dash = str(todaysdate_dt)

# MM/DD/YYYY
todaysdate_slash = todaysdate_dash.split("-")
todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# YYYYMMDD
todaysdate = todaysdate_dash.replace("-", "")

In [3]:
# This reads in a map of team name, codes, and the number Fangraphs uses in their URLs
team_map = pd.read_csv(os.path.join(baseball_path, "Utilities", "Team Map.csv"))

# We just need teams right now
team_map = team_map[['DKTEAM', 'BBREFTEAM', 'SFBBTEAM']]

### Lineups 

In [4]:
# Scrape team lineups
def scrape_lineups(date=todaysdate_dash, early=False):
    # Read in daily lineups from Baseball Monster
    url = "https://baseballmonster.com/Lineups.aspx?csv=1&d=" + date
    lineups = pd.read_csv(url)

    # Merge with team map to get proper team code
    lineups = lineups.merge(team_map, left_on='team code', right_on='SFBBTEAM', how='inner') 

    # Fill missings
    lineups[' mlb id'] = np.where(lineups[' player name'] == "Masataka Yoshida", 807799, lineups[' mlb id'])

    # Check missings
    lineups[' mlb id'].fillna(999999, inplace=True)
    for i in range(len(lineups)):
        if lineups[' mlb id'][i] == 999999:
            print(lineups[' player name'][i])

    # Determine if it's the first or second game of a double header
    lineups['count'] = lineups.groupby('BBREFTEAM').cumcount()+1
    lineups['game'] = np.where(lineups['count'] <= 10, 1, 2)
    lineups['games'] = lineups.groupby('BBREFTEAM')['game'].transform('max')
    
    # If it's early, keep the first game
    if early == True:
        lineups = lineups[lineups['game'] == 1]
    # If it's not, keep the last game
    else:
        lineups = lineups[lineups['game'] == lineups['games']]

    # Keep relevant variables
    lineups = lineups[[' mlb id', ' batting order', 'BBREFTEAM']]
    lineups.rename(columns={' mlb id': 'key_mlbam', ' batting order':'batting_order_fill'}, inplace=True)
    
    
    return lineups

In [5]:
# Save today's lineups as a CSV
todaysdate_dash = todaysdate[:4] + "-" + todaysdate[4:6] + "-" + todaysdate[6:]
lineups = scrape_lineups(date=todaysdate_dash, early=False)
lineups.to_csv(os.path.join(baseball_path, "A7. Matchups - 1. Lineups", "Lineups " + todaysdate_dash + ".csv"))

### DK Salaries

In [6]:
# This reads in the most recently downloaded salary file and saves it 
def create_dk_salaries(date=todaysdate):
    # Find all DK downloads
    dk_files = glob.glob(os.path.join(download_path, 'DKSalaries*.csv'))
    dk_files.sort(key=os.path.getmtime)

    latest = dk_files[-1]

    # Clean DK salaries for merge
    dk_salaries = pd.read_csv(os.path.join(download_path, latest))
    
    dk_salaries = dk_salaries[dk_salaries['Name'] != "Caleb Smith"] # There are two Ca Smiths on Pit. Kept Canaan
    
    dk_salaries['ID'] = np.where(dk_salaries['Name'] == "Shohei Ohtani", 134045, dk_salaries['ID'])
    
    dk_salaries.to_csv(os.path.join(baseball_path, "A7. Matchups - 1. Salaries", "DKSalaries_" + date + ".csv"))

In [7]:
create_dk_salaries(todaysdate)

In [8]:
# This reads in saves salary files and keeps relevant variables. It also creates a list of games.
def clean_dk_salaries(date=todaysdate):
    # Read in DK Salaries
    dk_salaries = pd.read_csv(os.path.join(baseball_path, "A7. Matchups - 1. Salaries", "DKSalaries_" + date + ".csv"))
    
    
    # Clean game info
    # For scraped, we already fix the names - make this better universally
    dk_salaries['Game Info'] = dk_salaries['Game Info'].replace({"CWS":"CHW", "KC": "KCR", "SD": "SDP", "SF":"SFG", "TB":"TBR", "WAS":"WSN", "@": "_", ":": "", "/": ""}, regex=True)
    dk_salaries['Game Info'] = dk_salaries['Game Info'].replace({"SDPP": "SDP", 'TBRR':'TBR', 'KCRR':'KCR'}, regex=True)
    dk_salaries['Game Info'] = dk_salaries['Game Info'].replace({"@": "_", ":": "", "/": ""}, regex=True)
    # Merge with team map to get baseball reference names
    dk_salaries = dk_salaries.merge(team_map, left_on='TeamAbbrev', right_on='DKTEAM', how='left')

    # Convert to Baseball Reference team code
    dk_salaries['TeamAbbrev'] = dk_salaries['BBREFTEAM']

    # Change Ohtani's number
    dk_salaries['ID'] = np.where((dk_salaries['Name'] == "Shohei Ohtani"), 134045, dk_salaries['ID'])
   
    # Clean names
    dk_salaries = name_clean(dk_salaries)

    # This is all we need to merge. 
    dk_salaries_cut = dk_salaries[['First2', 'Last5', 'BBREFTEAM', 'Salary', 'ID']]


    # We also want a separate game info df with teams, time, who's home, date
    game_info = dk_salaries[['Game Info']]

    # This is the list of all matchups
    matchups = game_info['Game Info'].unique()
    matchups = matchups.tolist()
    
    print(matchups)

    # Define a custom sorting key function that extracts the numbers from characters 17-20 in each string (the game time)
    # Note: AM games can mess this up a bit
    def sort_key(string):
        return int(string[17:21])

    
    try:
        matchups.remove('Postponed')
    except:
        pass
    try:
        matchups.remove('Cancelled')
    except:
        pass

    try:
        # Sort matchups on game time
        matchups.sort(key=sort_key)
    except:
        pass
    
    return dk_salaries_cut, matchups

In [9]:
def fill_ins(df):
    # Take first two characters of first name
    df['First2_fill'] = df['firstName'].str.slice(0,2)
    # And first 5 characters of last name
    df['Last5_fill'] = df['lastName'].str.slice(0,5)

    # Make lower case
    df['First2_fill'] = df['First2_fill'].str.lower()
    df['Last5_fill'] = df['Last5_fill'].str.lower()

    # Make string (this makes the f_remove_accents function work properly
    df['First2_fill'] = df['First2_fill'].astype(str) # this one is necessary
    df['Last5_fill'] = df['Last5_fill'].astype(str) # this one is not

    # Remove accents
    df['First2_fill'] = df.apply(lambda x: remove_accents(x['First2_fill']), axis=1)  # remove accents
    df['Last5_fill'] = df.apply(lambda x: remove_accents(x['Last5_fill']), axis=1)  # remove accents

    # Remove abnormal characters
    df['First2_fill'] = df['First2_fill'].str.replace('[^a-zA-Z0-9 ]', '')
    df['Last5_fill'] = df['Last5_fill'].str.replace('[^a-zA-Z0-9 ]', '')
    
    df['First2'].fillna(df['First2_fill'],inplace=True)
    df['Last5'].fillna(df['Last5_fill'],inplace=True)
    
    try:
        df['outs'].fillna(9, inplace=True)
        df['avgFaced'].fillna(15, inplace=True)
        df['starter_api'].fillna(1, inplace=True)
    except:
        pass
    
    df.drop(columns=['First2_fill', 'Last5_fill'], inplace=True)
    
    return df

### Matchup

In [10]:
def clean_matchups(df):
    # Add RP leverage to starting pitcher leverage (1 if starter)
    df['Leverage'] = np.where(df['starter'] == 1, 1, df['Leverage'])
    
    # Determine batting order
    df['batting_order'] = np.nan
    for i in range(9):
        df['batting_order'] = np.where(df['order'] == (i+1)*100, i+1, df['batting_order'])
    
    # Imputed flag
    try:
        df['imp'] = np.where(df['pa_b_long_r'] < 40, 1, 0)
    except:
        df['imp'] = np.where(df['pa_p_long_r'] < 40, 1, 0)
        
    # Delete unnamed columns
    df = df.loc[:,~df.columns.str.startswith('Unnamed')]
    
    return df

In [11]:
def create_matchups(date=todaysdate, backtest=False, early=False):        
    # Read in clean DK Salaries data
    dk_salaries_cut, matchups = clean_dk_salaries(date)
    
    # Scrape lineups 
    date_dash = date[:4] + "-" + date[4:6] + "-" + date[6:]
    print(date_dash)
    lineups = scrape_lineups(date=date_dash, early=early)
    
    daily_folder = "Daily" + date
    
    for matchup in matchups:
        # Create new folder with daily rosters
        matchup_folder = "Matchups" + date
        try:
            os.mkdir(os.path.join(baseball_path, "A7. Matchups - 2. Matchups", matchup_folder))
        except:
            pass
        
        away = matchup[0:3]
        home = matchup[4:7]
        
        away_file = away + date + ".xlsx"
        home_file = home + date + ".xlsx"
        
        # Read in away data
        away_batters = pd.read_excel(os.path.join(baseball_path, "A5. Stats - 2. Teams", daily_folder, away_file), sheet_name='Batters')
        away_pitchers = pd.read_excel(os.path.join(baseball_path, "A5. Stats - 2. Teams", daily_folder, away_file), sheet_name='Pitchers')
        
        # Read in home data
        home_batters = pd.read_excel(os.path.join(baseball_path, "A5. Stats - 2. Teams", daily_folder, home_file), sheet_name='Batters')
        home_pitchers = pd.read_excel(os.path.join(baseball_path, "A5. Stats - 2. Teams", daily_folder, home_file), sheet_name='Pitchers')
            
        
        # Fill in missings
        away_batters = fill_ins(away_batters)
        away_pitchers = fill_ins(away_pitchers)
        home_batters = fill_ins(home_batters)
        home_pitchers = fill_ins(home_pitchers)
    
        # Merge with DK Salaries
        away_batters = away_batters.merge(dk_salaries_cut, on=['First2', 'Last5', 'BBREFTEAM'], how='left')
        away_pitchers = away_pitchers.merge(dk_salaries_cut, on=['First2', 'Last5', 'BBREFTEAM'], how='left')
        
        home_batters = home_batters.merge(dk_salaries_cut, on=['First2', 'Last5', 'BBREFTEAM'], how='left')
        home_pitchers = home_pitchers.merge(dk_salaries_cut, on=['First2', 'Last5', 'BBREFTEAM'], how='left')
        
        # Create name variable so Sims reads in right objects
        away_batters['Name'] = away_batters['fullName']
        away_pitchers['Name'] = away_pitchers['fullName']
        home_batters['Name'] = home_batters['fullName']
        home_pitchers['Name'] = home_pitchers['fullName']
             
        # Clean
        away_batters = clean_matchups(away_batters)
        away_pitchers = clean_matchups(away_pitchers)
        
        home_batters = clean_matchups(home_batters)
        home_pitchers = clean_matchups(home_pitchers)   
        
        # Since they're getting fixed, we can make imp all 0, eventually get rid of it
        away_batters['imp'] = 0
        away_pitchers['imp'] = 0
        home_batters['imp'] = 0
        home_pitchers['imp'] = 0
        
        
        # Add lineups
        away_batters = away_batters.merge(lineups, on=['key_mlbam', 'BBREFTEAM'], how='left')
        home_batters = home_batters.merge(lineups, on=['key_mlbam', 'BBREFTEAM'], how='left')
        away_pitchers = away_pitchers.merge(lineups, on=['key_mlbam', 'BBREFTEAM'], how='left')
        home_pitchers = home_pitchers.merge(lineups, on=['key_mlbam', 'BBREFTEAM'], how='left')
        
        # If backtesting, use statsapi order
        if backtest == True:
            away_batters['batting_order'] = np.nan
            home_batters['batting_order'] = np.nan
            for i in range(1, 10):
                away_batters['batting_order'] = np.where(away_batters['order'] == i * 100, i, away_batters['batting_order'])
                home_batters['batting_order'] = np.where(home_batters['order'] == i * 100, i, home_batters['batting_order'])
                
        if backtest == False:
            # Fill in batting orders
            away_batters['batting_order_fill'].fillna("-1", inplace=True)
            away_batters['batting_order'] = away_batters['batting_order_fill'].astype('int')
            away_batters['batting_order'] = np.where(away_batters['batting_order'] == -1, np.nan, away_batters['batting_order'])
            away_batters.drop(columns={'batting_order_fill'}, inplace=True)

            home_batters['batting_order_fill'].fillna("-1", inplace=True)
            home_batters['batting_order'] = home_batters['batting_order_fill'].astype('int')
            home_batters['batting_order'] = np.where(home_batters['batting_order'] == -1, np.nan, home_batters['batting_order'])
            home_batters.drop(columns={'batting_order_fill'}, inplace=True)

            away_pitchers['batting_order'] = away_pitchers['batting_order_fill']
            away_pitchers['starter'] = np.where(away_pitchers['batting_order'] == "SP", 1, 0)
            away_pitchers['Leverage'] = np.where(away_pitchers['batting_order'] == "SP", 1, away_pitchers['Leverage'])
            away_pitchers.drop(columns={'batting_order_fill'}, inplace=True)

            home_pitchers['batting_order'] = home_pitchers['batting_order_fill']
            home_pitchers['starter'] = np.where(home_pitchers['batting_order'] == "SP", 1, 0)
            home_pitchers['Leverage'] = np.where(home_pitchers['batting_order'] == "SP", 1, home_pitchers['Leverage'])
            home_pitchers.drop(columns={'batting_order_fill'}, inplace=True)
        
        
        batter_info = ['fullName', 'firstName', 'lastName', 'Salary', 'batting_order', 'starter', 'Leverage', 'batSide', 'pitchHand', 
         'position', 'BBREFTEAM', 
         'ID', 'id', 'key_mlbam', 'key_fangraphs', 'key_bbref_minors', 'key_bbref', 
         'name_first', 'name_last', 'First2', 'Last5', 'Name', 'order',        
         'status', 'venue_id', 'game_date', 'game_type', 'game_num', 'summary', 
         'weather', 'wind', 'missing']
        
        batter_stat = [               
         'b_L', 'batSide_l', 
         'b1_b_l', 'b2_b_l', 'b3_b_l', 'hr_b_l', 'bb_b_l', 'hbp_b_l',               
         'so_b_l', 'lo_b_l', 'po_b_l', 'go_b_l', 'fo_b_l', 
         'woba_b_l', 'slg_b_l', 'obp_b_l', 'iso_b_l', 
         'to_left_b_l', 'to_middle_b_l', 'to_right_b_l',                      
         'hard_hit_b_l', 'totalDistance_b_l', 'launchSpeed_b_l', 
         'maxSpeed_b_l', 'maxSpin_b_l',
         'pa_b_l', 'ab_b_l', 
                       
         'b1_b_long_l', 'b2_b_long_l', 'b3_b_long_l', 'hr_b_long_l', 'bb_b_long_l', 'hbp_b_long_l',              
         'so_b_long_l', 'lo_b_long_l', 'po_b_long_l', 'go_b_long_l', 'fo_b_long_l', 
         'woba_b_long_l', 'slg_b_long_l', 'obp_b_long_l', 'iso_b_long_l',
         'to_left_b_long_l', 'to_middle_b_long_l', 'to_right_b_long_l',                      
         'hard_hit_b_long_l', 'totalDistance_b_long_l', 'launchSpeed_b_long_l', 
         'maxSpeed_b_long_l', 'maxSpin_b_long_l',                       
         'pa_b_long_l', 'ab_b_long_l', 
                       
                       
         'batSide_r', 
         'b1_b_r', 'b2_b_r', 'b3_b_r', 'hr_b_r', 'bb_b_r', 'hbp_b_r',
         'so_b_r', 'lo_b_r', 'po_b_r', 'go_b_r', 'fo_b_r', 
         'woba_b_r', 'slg_b_r', 'obp_b_r', 'iso_b_r', 
         'to_left_b_r', 'to_middle_b_r', 'to_right_b_r',                      
         'hard_hit_b_r', 'totalDistance_b_r', 'launchSpeed_b_r',
         'maxSpeed_b_r', 'maxSpin_b_r',        
         'pa_b_r', 'ab_b_r', 
                       
         'b1_b_long_r', 'b2_b_long_r', 'b3_b_long_r', 'hr_b_long_r', 'bb_b_long_r', 'hbp_b_long_r', 
         'so_b_long_r', 'lo_b_long_r', 'po_b_long_r', 'go_b_long_r', 'fo_b_long_r', 
         'woba_b_long_r', 'slg_b_long_r', 'obp_b_long_r','iso_b_long_r', 
         'to_left_b_long_r', 'to_middle_b_long_r', 'to_right_b_long_r',                      
         'hard_hit_b_long_r', 'totalDistance_b_long_r', 'launchSpeed_b_long_r',
         'maxSpeed_b_long_r', 'maxSpin_b_long_r',
         'pa_b_long_r', 'ab_b_long_r', 
                       
         'b1_rate', 'b2_rate', 'b3_rate', 'hr_rate', 'bb_rate', 'hbp_rate', 'so_rate', 'obp', 'slg', 'woba', 
         'sba_imp', 'sbr', 'sba_2b', 'sba_3b', 'sb_2b', 'sb_3b', 
         'imp']
        
        pitcher_info = ['fullName', 'firstName', 'lastName', 'Salary', 'batting_order', 'starter', 'Leverage', 'batSide', 'pitchHand', 
         'position', 'BBREFTEAM', 
         'ID', 'id', 'key_mlbam', 'key_fangraphs', 'key_bbref_minors', 'key_bbref',  
         'name_first', 'name_last', 'First2', 'Last5', 'Name', 'order',               
         'status', 'venue_id', 'game_date', 'game_type', 'game_num', 'summary', 
         'weather', 'wind', 'missing']
          
        pitcher_stat = [
         'p_L', 'pitchHand_l', 
         'b1_p_l', 'b2_p_l', 'b3_p_l', 'hr_p_l', 'bb_p_l', 'hbp_p_l', 
         'so_p_l', 'lo_p_l', 'po_p_l', 'go_p_l', 'fo_p_l', 
         'woba_p_l', 'slg_p_l', 'obp_p_l', 'iso_p_l', 
         'to_left_p_l', 'to_middle_p_l', 'to_right_p_l', 
         'hard_hit_p_l', 'totalDistance_p_l', 'launchSpeed_p_l',
         'maxSpeed_p_l', 'maxSpin_p_l', 
         'pa_p_l', 'ab_p_l',
                        
         'b1_p_long_l', 'b2_p_long_l', 'b3_p_long_l', 'hr_p_long_l', 'bb_p_long_l', 'hbp_p_long_l',                
         'so_p_long_l', 'lo_p_long_l', 'po_p_long_l', 'go_p_long_l', 'fo_p_long_l', 
         'woba_p_long_l', 'slg_p_long_l', 'obp_p_long_l', 'iso_p_long_l', 
         'to_left_p_long_l', 'to_middle_p_long_l', 'to_right_p_long_l', 
         'hard_hit_p_long_l', 'totalDistance_p_long_l', 'launchSpeed_p_long_l',
         'maxSpeed_p_long_l', 'maxSpin_p_long_l',                        
         'pa_p_long_l', 'ab_p_long_l', 
                        
         'pitchHand_r', 
         'b1_p_r', 'b2_p_r', 'b3_p_r', 'hr_p_r', 'bb_p_r', 'hbp_p_r',               
         'so_p_r', 'lo_p_r', 'po_p_r', 'go_p_r', 'fo_p_r', 
         'woba_p_r', 'slg_p_r', 'obp_p_r', 'iso_p_r', 
         'to_left_p_r', 'to_middle_p_r', 'to_right_p_r', 
         'hard_hit_p_r', 'totalDistance_p_r', 'launchSpeed_p_r',
         'maxSpeed_p_r', 'maxSpin_p_r', 
         'pa_p_r', 'ab_p_r', 
                        
         'b1_p_long_r', 'b2_p_long_r', 'b3_p_long_r', 'hr_p_long_r', 'bb_p_long_r', 'hbp_p_long_r',                
         'so_p_long_r', 'lo_p_long_r', 'po_p_long_r', 'go_p_long_r', 'fo_p_long_r', 
         'woba_p_long_r', 'slg_p_long_r', 'obp_p_long_r', 'iso_p_long_r', 
         'to_left_p_long_r', 'to_middle_p_long_r', 'to_right_p_long_r', 
         'hard_hit_p_long_r', 'totalDistance_p_long_r', 'launchSpeed_p_long_r',
         'maxSpeed_p_long_r', 'maxSpin_p_long_r',          
         'pa_p_long_r', 'ab_p_long_r', 
                        
         'H9', 'HR9', 'K9', 'BB9',                
         'outs', 'avgFaced',
         'imp']
        
        batter_list = batter_info + batter_stat
        pitcher_list = pitcher_info + pitcher_stat 
            
        # Keep relevant variables
        away_batters = away_batters[batter_list]
        home_batters = home_batters[batter_list]
        away_pitchers = away_pitchers[pitcher_list]
        home_pitchers = home_pitchers[pitcher_list]
        
        away_batters.sort_values('batting_order', inplace=True)
        home_batters.sort_values('batting_order', inplace=True)
        away_pitchers.sort_values('Leverage', inplace=True)
        home_pitchers.sort_values('Leverage', inplace=True)
        
        print(matchup)
        if away_batters['batting_order'].sum() != 45:
            print("The sum of the away team's batting order is {}.".format(away_batters['batting_order'].sum()))
        if home_batters['batting_order'].sum() != 45:
            print("The sum of the home team's batting order is {}.".format(home_batters['batting_order'].sum()))
        if 1 not in list(away_pitchers['Leverage']):
            print("The away team is missing a starting pitcher.")
        if 4 not in list(away_pitchers['Leverage']):
            print("The away team is missing a closer.")
        if 1 not in list(home_pitchers['Leverage']):
            print("The home team is missing a starting pitcher.")
        if 4 not in list(home_pitchers['Leverage']):
            print("The home team is missing a closer.")
        
        # Create file named after matchup
        matchup_file = matchup + ".xlsx"
        
        # Fill in missings with team averages
        for stat in batter_stat:
            try:
                stat_mean = away_batters[stat].mean()
                away_batters[stat].fillna(stat_mean, inplace=True)
            except:
                away_batters[stat].fillna("R", inplace=True)
                
            try:
                stat_mean = home_batters[stat].mean()
                home_batters[stat].fillna(stat_mean, inplace=True)
            except:
                home_batters[stat].fillna("R", inplace=True)
                
        for stat in pitcher_stat:
            try:
                stat_mean = away_pitchers[stat].mean()
                away_pitchers[stat].fillna(stat_mean, inplace=True)
            except:
                away_pitchers[stat].fillna("R", inplace=True)
                
            try:
                stat_mean = home_pitchers[stat].mean()
                home_pitchers[stat].fillna(stat_mean, inplace=True)
            except:
                home_pitchers[stat].fillna("R", inplace=True)
                        
                
        away_batters.fillna(-99, inplace=True)
        away_pitchers.fillna(-99, inplace=True)
        home_batters.fillna(-99, inplace=True)
        home_pitchers.fillna(-99, inplace=True)
        
        # Maybe print out players that are imputed and starting
        away_batters['imp'] = np.where(away_batters['name_last'] == -99, 1, 0)
        away_pitchers['imp'] = np.where(away_pitchers['name_last'] == -99, 1, 0)
        home_batters['imp'] = np.where(home_batters['name_last'] == -99, 1, 0)
        home_pitchers['imp'] = np.where(home_pitchers['name_last'] == -99, 1, 0)
        
        # Write to Excel
        away_batters.to_excel(os.path.join(baseball_path, "A7. Matchups - 2. Matchups", matchup_folder, matchup_file), sheet_name="AwayBatters", engine='openpyxl')

        with pd.ExcelWriter(os.path.join(baseball_path, "A7. Matchups - 2. Matchups", matchup_folder, matchup_file), mode='a', engine='openpyxl') as writer:  
            away_pitchers.to_excel(writer, sheet_name='AwayPitchers')

        with pd.ExcelWriter(os.path.join(baseball_path, "A7. Matchups - 2. Matchups", matchup_folder, matchup_file), mode='a', engine='openpyxl') as writer:  
            home_batters.to_excel(writer, sheet_name='HomeBatters')

        with pd.ExcelWriter(os.path.join(baseball_path, "A7. Matchups - 2. Matchups", matchup_folder, matchup_file), mode='a', engine='openpyxl') as writer:  
            home_pitchers.to_excel(writer, sheet_name='HomePitchers')

### Run One

In [12]:
# Create daily matchups:
# Note: early should default to false, although it only matters in double headers
create_matchups(date=todaysdate, backtest=False, early=False)

['TOR_BAL 06142023 0705PM ET', 'LAA_TEX 06142023 0805PM ET', 'NYY_NYM 06142023 0710PM ET', 'WSN_HOU 06142023 0810PM ET', 'CHW_LAD 06142023 1010PM ET', 'TBR_OAK 06142023 0940PM ET', 'PHI_ARI 06142023 0940PM ET', 'MIA_SEA 06142023 0940PM ET', 'CLE_SDP 06142023 0940PM ET', 'PIT_CHC 06142023 0805PM ET', 'COL_BOS 06142023 0710PM ET', 'CIN_KCR 06142023 0810PM ET']
2023-06-14
TOR_BAL 06142023 0705PM ET
NYY_NYM 06142023 0710PM ET
COL_BOS 06142023 0710PM ET
LAA_TEX 06142023 0805PM ET
PIT_CHC 06142023 0805PM ET
WSN_HOU 06142023 0810PM ET
CIN_KCR 06142023 0810PM ET
TBR_OAK 06142023 0940PM ET
PHI_ARI 06142023 0940PM ET
MIA_SEA 06142023 0940PM ET
CLE_SDP 06142023 0940PM ET
CHW_LAD 06142023 1010PM ET


### Run All

In [13]:
# for filename in os.listdir(r"C:\Users\james\Documents\MLB\Data\A7. Matchups - 1. Salaries"): 
#     # 2023 
#     if filename.endswith(".csv") and filename.startswith("DKSalaries_2023"):
#         # Pull out date
#         date = filename[11:19]
#         print(date)
#         try:
#             create_matchups(date=date, backtest=False, early=False)
#         except:
#             print("It ain't work")

In [14]:
print("Code was last run on: {} at {}.".format(datetime.date.today(), datetime.datetime.now().strftime("%H:%M:%S")))

Code was last run on: 2023-06-14 at 23:46:59.
